<a href="https://colab.research.google.com/github/HAVIGILI/OOD-Co-Act/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip uninstall -y accelerate

%pip install numpy==1.24.3
%pip install --quiet gdown

# Install OpenMIM tool
%pip install -U openmim

# Install a compatible PyTorch (for example, 2.0.0+cu118)
%pip install torch==2.0.0+cu118 torchvision==0.15.0+cu118 -f https://download.pytorch.org/whl/cu118/torch_stable.html

%pip install torchmetrics

# Then install mmcv-full pre-built for that combination:
!mim install "mmcv-full==1.7.2" -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html

Looking in links: https://download.pytorch.org/whl/cu118/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 926.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [2]:
!git clone -b 0.x --depth 1 https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .


Cloning into 'mmsegmentation'...
remote: Enumerating objects: 1405, done.
remote: Counting objects: 100% (1405/1405), done.
remote: Compressing objects: 100% (829/829), done.
remote: Total 1405 (delta 726), reused 892 (delta 556), pack-reused 0 (from 0)
Receiving objects: 100% (1405/1405), 9.06 MiB | 32.54 MiB/s, done.
Resolving deltas: 100% (726/726), done.
/content/mmsegmentation
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.8/648.8 kB 12.0 MB/s eta 0:00:00
  Running setup.py develop for mmsegmentation


In [3]:
import os
import torch
from mmseg.apis import init_segmentor, inference_segmentor
import mmcv
from IPython.display import Image, display
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
# Fishyscapes importet below. Keep this commented out

# RoadAnomaly21:
# !unzip -q /content/dataset_RoadAnomalyTrack.zip -d RoadAnomaly
# !unzip -q /content/gtFine_trainvaltest.zip -d gtFine_trainvaltest
# !wget -q http://wwwlehre.dhbw-stuttgart.de/~sgehrig/lostAndFoundDataset/leftImg8bit.zip
# !unzip -q leftImg8bit.zip -d /content/dataset_root/
# !wget -q http://robotics.ethz.ch/~asl-datasets/Fishyscapes/fishyscapes_lostandfound.zip
# !unzip -q fishyscapes_lostandfound.zip -d /content/dataset_root/

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# To get fishyscapes ground truth and pictures.

LEFTZIP_ID = "1BihdEUYOCRKpLfo8VvajTbuUtV3RyFtH"
FISHYZIP_ID = "1aR_qSjuykKWuKvM1dayecfb37cNYfZC3"

# download both
!gdown https://drive.google.com/uc?id=$LEFTZIP_ID -O leftImg8bit.zip
!gdown https://drive.google.com/uc?id=$FISHYZIP_ID -O fishyscapes.zip

# unzip into the exact paths your code expects
!unzip -q leftImg8bit.zip -d /content/drive/MyDrive/
!unzip -q fishyscapes.zip -d /content/drive/MyDrive/

images_root      = "/content/drive/MyDrive/leftImg8bit"
annotations_root = "/content/drive/MyDrive/fishyscapes_lostandfound"

Downloading...
From (original): https://drive.google.com/uc?id=1BihdEUYOCRKpLfo8VvajTbuUtV3RyFtH
From (redirected): https://drive.google.com/uc?id=1BihdEUYOCRKpLfo8VvajTbuUtV3RyFtH&confirm=t&uuid=c6d34d48-2cc7-4979-980e-2361f7406e7e
To: /content/mmsegmentation/leftImg8bit.zip
100% 5.80G/5.80G [00:31<00:00, 185MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aR_qSjuykKWuKvM1dayecfb37cNYfZC3
To: /content/mmsegmentation/fishyscapes.zip
100% 735k/735k [00:00<00:00, 158MB/s]
replace /content/drive/MyDrive/leftImg8bit/train/03_Hanns_Klemm_Str_19/03_Hanns_Klemm_Str_19_000003_000100_leftImg8bit.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace /content/drive/MyDrive/fishyscapes_lostandfound/0001_04_Maurener_Weg_8_000000_000090_labels.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
# Matching images with ground truth for fishyscapes.

import os

matched_annotations = []
matched_images = []
annotated_images = [f[5:].replace("_labels.png", "_leftImg8bit.png") for f in os.listdir(annotations_root)]
ood_annotations_paths = [os.path.join(annotations_root, f) for f in os.listdir(annotations_root)]

# Walk through the image directory
for dir_path, dir_names, filenames in os.walk(images_root):
        for filename in filenames:
            if filename in annotated_images:
                index = annotated_images.index(filename)
                annotation_path = ood_annotations_paths[index]
                image_path = os.path.join(dir_path, filename)
                matched_annotations.append(annotation_path)
                matched_images.append(image_path)
print("Matched pairs:")
for annotation, image in zip(matched_annotations, matched_images):
    print("annotation", annotation, "matched with", image)


Matched pairs:
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0095_13_Elly_Beinhorn_Str_000002_000120_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000002_000120_leftImg8bit.png
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0088_13_Elly_Beinhorn_Str_000000_000290_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000000_000290_leftImg8bit.png
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0092_13_Elly_Beinhorn_Str_000001_000230_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000001_000230_leftImg8bit.png
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0089_13_Elly_Beinhorn_Str_000001_000080_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000001_000080_leftImg8bit.png
annotation /content/drive/MyDrive/fishysc

In [8]:
# Initialize the segmentation model (DeepLabV3+ pretrained on Cityscapes)
config_file = 'configs/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes.py'
checkpoint_file = ('https://download.openmmlab.com/mmsegmentation/v0.5/'
                   'deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/'
                   'deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth')
model = init_segmentor(config_file, checkpoint_file, device='cuda:0' if torch.cuda.is_available() else 'cpu')

/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from http path: https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth


Downloading: "https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth
100%|██████████| 239M/239M [00:07<00:00, 35.3MB/s]


In [9]:
# calibration.py – Calibrator for LINe and pair-wise co-activation stats
# =====================================================================
# Author: <your-name>
# This module gathers statistics that are later used for OOD detection.
# Everything is pure PyTorch, no heavy dependencies besides MMSeg.

from __future__ import annotations

from typing import List, Dict, Any

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from mmseg.apis import inference_segmentor

# ----------------------------------------------------------------------
#  A fixed raw-to-train LUT for the Cityscapes labels
# ----------------------------------------------------------------------
_RAW2TRAIN: dict[int, int] = {
    7: 0,  8: 1, 11: 2, 12: 3, 13: 4, 17: 5, 19: 6, 20: 7,
    21: 8, 22: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14,
    28: 15, 31: 16, 32: 17, 33: 18,
}
_LUT = np.full(256, 255, np.uint8)          # 255 = “ignore”
for raw_id, train_id in _RAW2TRAIN.items():
    _LUT[raw_id] = train_id


# ======================================================================
#  Calibrator
# ======================================================================
class Calibrator:
    """Collect LINe tensors *and* pair-wise co-activation statistics."""

    def __init__(
        self,
        mmseg_model: nn.Module,            # ← pass a *ready* MMSeg model
        device: str | None = None,
        n_classes: int = 19,
        n_channels: int = 512,
    ) -> None:
        self.model = mmseg_model
        self.dev   = device or next(mmseg_model.parameters()).device
        self.K, self.C = n_classes, n_channels

        # ---------- LINe tensors ----------
        self.contrib = torch.zeros(self.K, self.C, device=self.dev)
        self.avg_act = torch.zeros(self.K, self.C, device=self.dev)

        # ---------- per-class accumulators ----------
        sz = (self.K, self.C, self.C)
        self.co_bin_count   = torch.zeros(sz, device=self.dev)
        self.co_wt_sum      = torch.zeros(sz, device=self.dev)
        self.co_wt_sum_both = torch.zeros(sz, device=self.dev)
        self.pixel_counter  = torch.zeros(self.K, device=self.dev)

        # ---------- global accumulators ----------
        self.co_bin_all     = torch.zeros(self.C, self.C, device=self.dev)
        self.co_wt_all      = torch.zeros(self.C, self.C, device=self.dev)
        self.co_wt_both_all = torch.zeros(self.C, self.C, device=self.dev)
        self.pixel_all      = torch.tensor(0.0, device=self.dev)

    # ------------------------------------------------------------------
    #  Main dataset pass
    # ------------------------------------------------------------------
    def run(self, images: List[str], gt_paths: List[str]) -> None:
        """Iterate over *images* and *gt_paths* to fill all statistics."""

        hook_buf: dict[str, torch.Tensor] = {}

        def _cache_feat(_, inp, __):
            hook_buf["feat"] = inp[0].detach()          # [1,C,Hf,Wf]

        self.model.decode_head.conv_seg.register_forward_hook(_cache_feat)
        w = self.model.decode_head.conv_seg.weight.data.squeeze(-1).squeeze(-1)  # [K,C]

        for idx, (img_p, gt_p) in enumerate(zip(images, gt_paths), 1):
            if idx % 25 == 0:
                print(f"[Calibrator] processed {idx}/{len(images)} images …")

            _ = inference_segmentor(self.model, img_p)           # runs model
            feat = hook_buf.pop("feat").squeeze(0)               # [C,Hf,Wf]
            Hf, Wf = feat.shape[1:]

            # ------------------ prepare ground truth ---------------------
            gt_raw   = cv2.imread(gt_p, cv2.IMREAD_GRAYSCALE)
            gt_train = _LUT[gt_raw]                              # 0-18 / 255
            valid    = (gt_train != 255).astype(np.uint8)

            gt_t = torch.from_numpy(gt_train)[None, None].float().to(self.dev)
            gt_t = F.interpolate(gt_t, size=(Hf, Wf), mode="nearest").long().squeeze()

            val_t = torch.from_numpy(valid)[None, None].float().to(self.dev)
            val_t = F.interpolate(val_t, size=(Hf, Wf), mode="nearest").bool().squeeze()

            # ------------------ accumulate statistics -------------------
            for cls in range(self.K):
                mask = (gt_t == cls) & val_t
                if not mask.any():
                    continue

                vecs = feat.permute(1, 2, 0)[mask]      # [N,C]
                binm = (vecs > 0).float()               # [N,C]
                n    = vecs.size(0)

                # per-class
                self.co_bin_count[cls]   += binm.T @ binm
                self.co_wt_sum[cls]      += vecs.T @ vecs
                self.co_wt_sum_both[cls] += (vecs * binm).T @ (vecs * binm)
                self.pixel_counter[cls]  += n

                # global
                self.co_bin_all     += binm.T @ binm
                self.co_wt_all      += vecs.T @ vecs
                self.co_wt_both_all += (vecs * binm).T @ (vecs * binm)
                self.pixel_all      += n

                # LINe tensors
                self.contrib[cls] += (vecs.sum(0) * w[cls]).abs()
                self.avg_act[cls] += (vecs > 0).sum(0)

        # ------------------ normalisation -------------------------------
        eps = 1e-9
        pix_cls = self.pixel_counter.clamp_min(eps).view(self.K, 1)
        self.contrib /= pix_cls
        self.avg_act /= pix_cls

        pc = self.pixel_counter.view(self.K, 1, 1) + eps
        self.freq_bin     = self.co_bin_count   / pc
        self.mean_wt      = self.co_wt_sum      / pc
        self.mean_wt_both = self.co_wt_sum_both / (self.co_bin_count + eps)

        diag = torch.diagonal(self.co_bin_count, dim1=1, dim2=2)
        cnt_any = diag.unsqueeze(2) + diag.unsqueeze(1) - self.co_bin_count
        self.mean_wt_any = self.co_wt_sum / (cnt_any + eps)

        self.freq_bin_all     = self.co_bin_all     / (self.pixel_all + eps)
        self.mean_wt_all      = self.co_wt_all      / (self.pixel_all + eps)
        self.mean_wt_both_all = self.co_wt_both_all / (self.co_bin_all + eps)

    # ------------------------------------------------------------------
    #  (De)serialisation helpers
    # ------------------------------------------------------------------
    def state_dict(self) -> Dict[str, Any]:
        """Return everything needed to recreate the calibrator later."""
        return {
            "model_weights":       self.model.state_dict(),
            "contrib":             self.contrib.cpu(),
            "avg_act":             self.avg_act.cpu(),
            "freq_bin":            self.freq_bin.cpu(),
            "mean_wt":             self.mean_wt.cpu(),
            "mean_wt_both":        self.mean_wt_both.cpu(),
            "mean_wt_any":         self.mean_wt_any.cpu(),
            "freq_bin_all":        self.freq_bin_all.cpu(),
            "mean_wt_all":         self.mean_wt_all.cpu(),
            "mean_wt_both_all":    self.mean_wt_both_all.cpu(),
        }

    def save(self, path: str) -> None:
        torch.save(self.state_dict(), path)
        print(f"[Calibrator] saved → {path}")

    # ------------------------------------------------------------------
    @classmethod
    def load(
        cls,
        path: str,
        mmseg_cfg: str,
        mmseg_ckpt: str,
        device: str | None = None,
    ) -> "Calibrator":
        """Recreate a Calibrator from disk plus a fresh MMSeg model."""
        from mmseg.apis import init_segmentor

        raw_model = init_segmentor(mmseg_cfg, mmseg_ckpt,
                                   device=device or 'cpu')
        obj  = cls(raw_model, device)
        data = torch.load(path, map_location=obj.dev)

        for k, v in data.items():
            if k == "model_weights":
                continue
            setattr(obj, k, v.to(obj.dev))

        raw_model.load_state_dict(data["model_weights"], strict=False)
        print(f"[Calibrator] loaded ← {path}")
        return obj


In [10]:
# @title
import copy
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import torch
import torch.nn.functional as F
from mmseg.apis import inference_segmentor
from typing import Tuple
from sklearn.metrics import (roc_auc_score, roc_curve,
                             average_precision_score,
                             precision_recall_curve)
# För TorchMetrics-baserade metoder
from torchmetrics.classification import BinaryAUROC, BinaryAveragePrecision, BinaryROC
import os
from typing import List
from matplotlib.colors import ListedColormap

#########################################################
#  Inference wrapper that applies LINe-style masking    #
#  + four pairwise-co-activation variants               #
#########################################################
class AnomalyDetector:
    def __init__(self,
                 anomaly_images,
                 anomaly_IDs,
                 model,
                 state_dict):
        # ----- unpack model & LINe state -----
        self.model                         = model
        self.device = next(model.parameters()).device
        self.contribution_tensor           = state_dict['contrib'].to(self.device)
        self.average_activations_per_class = state_dict['avg_act'].to(self.device)

        # ----- thresholds -----
        self.activation_clipping     = 1000
        self.activation_pruning      = 0
        self.weight_pruning          = 0
        self.temperature_co          = 1
        self.temperature_line        = 1
        self.inverse_convert_to_ones = False
        self.wt_threshold            = 0
        self.wt_u_threshold          = 1
        self.co_weighted             = 1
        self.max_pool                = 0
        self.use_elsewhere_map       = None
        self.weight_mode             = "softmax"
        self.make_feat_binary        = False
        self.c_u_ratio               = False

        # ----- unpack coact stats -----
        self.mean_freq     = state_dict['freq_bin'].to(self.device)      # P[aᵢ>0 ∧ aⱼ>0]     freq_bin is not binary.
        self.mean_wt       = state_dict['mean_wt'].to(self.device)      # E[aᵢ·aⱼ]
        self.mean_wt_both  = state_dict['mean_wt_both'].to(self.device)  # E[aᵢ·aⱼ | aᵢ>0∧aⱼ>0]
        self.mean_wt_any   = state_dict['mean_wt_any'].to(self.device)   # E[aᵢ·aⱼ | aᵢ>0∨aⱼ>0]

        # These are not used, yet. They are not class-specific. Any needs an "all" as well
        self.freq_bin_all     = state_dict['freq_bin'].to(self.device)
        self.mean_wt_all       = state_dict['mean_wt'].to(self.device)
        self.mean_wt_all       = state_dict['mean_wt_both'].to(self.device)

        self.common_freq   = None
        self.common_wt     = None
        self.common_both   = None
        self.common_any    = None
        self.uncommon_freq = None
        self.uncommon_wt   = None
        self.uncommon_both = None
        self.uncommon_any  = None

        # ----- prepare masks for LINe -----
        self.activation_mask     = torch.zeros(19, 512, device=self.device)
        self.common_neurons_mask = torch.zeros(19, 512, device=self.device)
        self.weight_mask         = torch.zeros(19, 19, 512, device=self.device)

        self.model_output      = None
        self.line_model_output = [0] * 19
        self.model_IDs_output  = None
        self.nr_of_activations = None
        self.input             = None
        self.feature_map       = None

        # ----- data & plotting -----
        self.anomaly_images = anomaly_images
        self.anomaly_IDs    = anomaly_IDs
        self.images_to_be_plotted = []
        self.plot_pixel_activations = False
        self.plot_pair_ratios = False


        # ----- fixed conv weight tensor -----
        self.weight_tensor = (self.model.decode_head.conv_seg.weight
                              .data.squeeze(-1).squeeze(-1))

        self.setup_masks()
        self.id_dot = (10, 10)
        self.ood_dot = (10, 10)

        # ----- maps containers -----
        self.gt_maps            = []
        self.raw_line_maps      = []
        self.raw_softmax_maps   = []
        self.raw_logitmin_maps  = []
        self.raw_logitmax_maps  = []
        self.raw_nract_maps     = []
        self.raw_euclidian_maps = []
        self.raw_coact_bin      = []
        self.raw_coact_wt       = []
        self.raw_coact_both     = []
        self.raw_coact_any      = []

        self.co_blur_ksize      = 0
        self.clips              = (0,0)
        self.co_sigma           = 0
        self.baseline_and_line_blur_ksize = (0,0)
        self.baseline_and_line_sigma = 0

        # ----- register hook on conv_seg -----
        self.model.decode_head.conv_seg._forward_hooks.clear()
        self.model.decode_head.conv_seg.register_forward_hook(
            self.hook
        )

    def hook(self, module, inp, out):
        line_input = inp[0].clone()
        self.input       = line_input
        self.feature_map = line_input.detach()

        # standard conv
        pred_logits = F.conv2d(line_input,
                                module.weight,
                                module.bias,
                                stride=module.stride,
                                padding=module.padding,
                                dilation=module.dilation,
                                groups=module.groups)
        pred_ids = torch.argmax(pred_logits, dim=1)

        # LINe per class (masking + conv). It is possible to remove bias
        for cls in range(19):
            mask_c = self.activation_mask[cls].view(1,512,1,1)
            in2 = line_input.clamp(max=self.activation_clipping) * mask_c
            w_m = self.weight_mask[cls].view_as(module.weight)
            upd_w = module.weight * w_m
            self.line_model_output[cls] = F.conv2d(
                in2, upd_w, module.bias,
                stride=module.stride,
                padding=module.padding,
                dilation=module.dilation,
                groups=module.groups
            )

        self.nr_of_activations = (line_input.squeeze(0)>0).sum(dim=0)
        self.model_output      = pred_logits
        self.model_IDs_output  = pred_ids
        return out

    def ood_inference(self):
        """
        Run inference, and get raw maps for all methods.
        """
        self.setup_masks()
        for i, (img_p, gt_p) in enumerate(zip(self.anomaly_images, self.anomaly_IDs)):
            result = inference_segmentor(self.model, img_p)
            gt     = cv2.imread(gt_p, cv2.IMREAD_GRAYSCALE)

            nr_act    = self._calculate_nr_activation_scores(gt)
            line      = self._calculate_line_scores(gt)
            softmax   = self._calculate_softmax_scores(gt)
            logitmin  = self._calculate_logitmin_scores(gt)
            logitmax  = self._calculate_logitmax_scores(gt)
            euclidian = self._calculate_euclidean_scores(gt)
            coact_any  = self._calculate_coactivation_score(gt, self.common_any, self.uncommon_any)
            coact_bin  =  self._calculate_coactivation_score(gt, self.common_freq, self.uncommon_freq)
            coact_wt   =  self._calculate_coactivation_score(gt, self.common_wt, self.uncommon_wt)
            coact_both = self._calculate_coactivation_score(gt, self.common_both, self.uncommon_both)

            # six original maps
            self.raw_nract_maps.append(     nr_act)
            self.raw_line_maps.append(      line)
            self.raw_softmax_maps.append(   softmax)
            self.raw_logitmin_maps.append(  logitmin)
            self.raw_logitmax_maps.append(  logitmax)
            self.raw_euclidian_maps.append( euclidian)

            # four coact variants
            self.raw_coact_bin.append(      coact_bin)
            self.raw_coact_wt.append(       coact_wt)
            self.raw_coact_both.append(     coact_both)
            self.raw_coact_any.append(      coact_any)

            self.gt_maps.append(gt)
            if i in self.images_to_be_plotted:
                self.plot_maps(result, img_p, gt,
                              self.raw_line_maps[-1],
                              self.raw_softmax_maps[-1],
                              self.raw_logitmin_maps[-1],
                              self.raw_logitmax_maps[-1],
                              self.raw_euclidian_maps[-1],
                              self.raw_nract_maps[-1],
                              self.raw_coact_bin[-1],
                              self.raw_coact_wt[-1],
                              self.raw_coact_both[-1],
                              self.raw_coact_any[-1])

    def get_ood_score_lists(self):
        # flatten GT and filter ignore
        all_gt = np.concatenate([m.ravel() for m in self.gt_maps])
        valid  = (all_gt != 255)

        # collect all ten methods
        scores = {
            "ood_scores_line":          np.concatenate([m.ravel() for m in self.raw_line_maps])[valid],
            "ood_scores_softmax":       np.concatenate([m.ravel() for m in self.raw_softmax_maps])[valid],
            "ood_scores_logitmin":      np.concatenate([m.ravel() for m in self.raw_logitmin_maps])[valid],
            "ood_scores_logitmax":      np.concatenate([m.ravel() for m in self.raw_logitmax_maps])[valid],
            "ood_scores_nr_activation": np.concatenate([m.ravel() for m in self.raw_nract_maps])[valid],
            "ood_scores_euclidian":     np.concatenate([m.ravel() for m in self.raw_euclidian_maps])[valid],
            "ood_scores_coact_bin":     np.concatenate([m.ravel() for m in self.raw_coact_bin])[valid],
            "ood_scores_coact_wt":      np.concatenate([m.ravel() for m in self.raw_coact_wt])[valid],
            "ood_scores_coact_both":    np.concatenate([m.ravel() for m in self.raw_coact_both])[valid],
            "ood_scores_coact_any":     np.concatenate([m.ravel() for m in self.raw_coact_any])[valid],
        }
        return all_gt[valid], scores

    # --------------------------------------------------------------
    #  Score-map calculators
    # --------------------------------------------------------------
    def _calculate_line_scores(self, labels):
        ood = np.zeros((256, 512))
        for ID in range(19):
            m = (self.model_IDs_output == ID).squeeze(0).cpu().numpy()
            logits = self.line_model_output[ID].squeeze(0)
            ood[m] = -(torch.logsumexp(logits / self.temperature_line, dim=0) * self.temperature_line).cpu().numpy()[m]

        self.use_elsewhere_map = ood #This is done to combine methods first, and do blurring as well, but after.
        ood = cv2.resize(ood, labels.shape[::-1], interpolation=cv2.INTER_NEAREST)
        return cv2.GaussianBlur(ood, self.baseline_and_line_blur_ksize, self.baseline_and_line_sigma) if self.baseline_and_line_blur_ksize != (0, 0) else ood

    def _calculate_logitmax_scores(self, labels):
        ood = -torch.max(self.model_output, dim=1).values.squeeze(0).cpu().numpy()
        ood = cv2.resize(ood, labels.shape[::-1], interpolation=cv2.INTER_NEAREST)
        return cv2.GaussianBlur(ood, self.baseline_and_line_blur_ksize, self.baseline_and_line_sigma) if self.baseline_and_line_blur_ksize != (0, 0) else ood

    def _calculate_softmax_scores(self, labels):
        sm  = F.softmax(self.model_output, dim=1)
        ood = -torch.max(sm, dim=1).values.squeeze(0).cpu().numpy()
        ood = cv2.resize(ood, labels.shape[::-1], interpolation=cv2.INTER_NEAREST)
        return cv2.GaussianBlur(ood, self.baseline_and_line_blur_ksize, self.baseline_and_line_sigma) if self.baseline_and_line_blur_ksize != (0, 0) else ood

    def _calculate_logitmin_scores(self, labels):
        ood = -torch.min(self.model_output, dim=1).values.squeeze(0).cpu().numpy()
        ood = cv2.resize(ood, labels.shape[::-1], interpolation=cv2.INTER_NEAREST)
        return cv2.GaussianBlur(ood, self.baseline_and_line_blur_ksize, self.baseline_and_line_sigma) if self.baseline_and_line_blur_ksize != (0, 0) else ood

    def _calculate_nr_activation_scores(self, labels):
        ood = np.zeros((256, 512))
        for ID in range(19):
            act_imp = ((self.input > 0) *
                       self.common_neurons_mask[ID].view(1, 512, 1, 1))
            m = (self.model_IDs_output == ID).squeeze(0).cpu().numpy()
            cur = act_imp.sum(dim=1).squeeze(0).cpu().numpy()
            ood[m] = cur[m]
        #self.use_elsewhere_map = ood
        ood = cv2.resize(ood, labels.shape[::-1], interpolation=cv2.INTER_NEAREST)
        return cv2.GaussianBlur(ood, self.baseline_and_line_blur_ksize, self.baseline_and_line_sigma) if self.baseline_and_line_blur_ksize != (0, 0) else ood

    def _calculate_euclidean_scores(self, labels, eps=1e-9):
        _, _, Hf, Wf = self.input.shape
        H_img, W_img = labels.shape
        dev          = self.input.device

        avg = (self.contribution_tensor /
                (self.weight_tensor.abs() + eps)).to(dev)

        feat  = self.input.squeeze(0).permute(1,2,0).reshape(-1, 512)
        cls   = self.model_IDs_output.squeeze(0).reshape(-1)
        dist  = (feat - avg[cls]).pow(2).sum(1).sqrt()

        ood = dist.view(Hf, Wf).cpu().numpy()

        # If using lowest distance instead of prediction:

        # _, _, Hf, Wf = self.input.shape
        # H_img, W_img = labels.shape
        # dev          = self.input.device

        # # avg: [K, C], K = number of classes (e.g. 19), C = feature dim (512)
        # avg = (self.contribution_tensor / (self.weight_tensor.abs() + eps)).to(dev)

        # # 1) flatten feature map → [N, C]
        # feat = (
        #     self.input
        #         .squeeze(0)                    # [C, Hf, Wf]
        #         .permute(1, 2, 0)              # [Hf, Wf, C]
        #         .reshape(-1, 512)              # [N, C]
        #         .to(dev)
        # )

        # # 2) compute squared distances to every class-mean → [N, K]
        # #    feat.unsqueeze(1): [N, 1, C], avg.unsqueeze(0): [1, K, C]
        # dists_sq = (feat.unsqueeze(1) - avg.unsqueeze(0)).pow(2).sum(dim=2)  # [N, K]

        # # 3) take sqrt of the minimum over classes → [N]
        # min_dist = dists_sq.min(dim=1).values.sqrt()  # [N]

        # # 4) reshape back to [Hf, Wf], resize, and blur
        # ood = min_dist.view(Hf, Wf).cpu().numpy()

        ood = cv2.resize(ood, labels.shape[::-1], interpolation=cv2.INTER_NEAREST)
        return cv2.GaussianBlur(ood, self.baseline_and_line_blur_ksize, self.baseline_and_line_sigma) if self.baseline_and_line_blur_ksize != (0, 0) else ood

    def setup_masks(self):
        # initialize accumulators for per-class weighting
        weighted_freq  = torch.zeros_like(self.mean_freq)
        weighted_wt    = torch.zeros_like(self.mean_wt)
        weighted_both  = torch.zeros_like(self.mean_wt_both)
        weighted_any   = torch.zeros_like(self.mean_wt_any)


        # Weight matrix used for experimenting with weights for the Co-act matrix. Can be ignored
        weight_co_tensor = torch.zeros_like(self.mean_freq)
        for ID in range(19):
            w_raw = self.weight_tensor[ID].view(-1,1)
            w_pos = torch.relu(w_raw)          # positives
            w_neg = torch.relu(-w_raw)         # negatives as positives
            w_abs = w_raw.abs()                # magnitude
            w_raw = self.weight_tensor[ID].view(-1,1)
            pos   = (w_raw > 0).float()   # 1 where wᵢ>0
            neg   = (w_raw < 0).float()   # 1 where wᵢ<0
            # 1) only positive–positive pairs
            mat_pospos = pos @ pos.T        # ones only where wᵢ>0 AND wⱼ>0

            # 2) only negative–negative pairs
            mat_negneg = neg @ neg.T        # ones only where wᵢ<0 AND wⱼ<0

            # 3) only negative–positive (i negative, j positive)
            mat_negpos = neg @ pos.T        # ones only where wᵢ<0 AND wⱼ>0

            # (optionally the flip: positive–negative)
            mat_posneg = pos @ neg.T        # ones only where wᵢ>0 AND wⱼ<0


            if   self.co_weighted ==  0:
              weight_matrix = torch.ones_like(self.mean_freq[ID])
            elif self.co_weighted ==  1:
                weight_matrix = w_pos @ w_pos.T
            elif self.co_weighted ==  2:
                weight_matrix = w_pos + w_pos.T
            elif self.co_weighted ==  3:
                weight_matrix = w_neg @ w_neg.T
            elif self.co_weighted ==  4:
                weight_matrix = w_neg + w_neg.T
            elif self.co_weighted ==  5:
                weight_matrix = w_raw @ w_raw.T
            elif self.co_weighted ==  6:
                weight_matrix = -(w_raw + w_raw.T)
            elif self.co_weighted ==  7:
                weight_matrix = w_abs @ w_abs.T
            elif self.co_weighted ==  8:
                weight_matrix = w_abs + w_abs.T
            elif self.co_weighted ==  9:
                weight_matrix = w_pos + w_neg.T
            elif self.co_weighted == 10:
                weight_matrix = w_neg + w_pos.T
            elif self.co_weighted == 11:
                weight_matrix = mat_pospos
            elif self.co_weighted == 12:
                weight_matrix = mat_negneg
            elif self.co_weighted == 13:
                weight_matrix = mat_negpos
            elif self.co_weighted == 14:
                weight_matrix = mat_posneg
            elif   self.co_weighted == 15:
                # broadcast: [-w_neg] is [C,1], w_pos.T is [1,C] → result [C,C]
                weight_matrix = -(-w_neg + w_pos.T)
            elif self.co_weighted == 16:
                # sum of absolute weights, but keep the original sign of (w_i + w_j)
                w_sum = w_raw + w_raw.T                # [C,C]
                m     = w_abs + w_abs.T                # [C,C]
                weight_matrix = -m * torch.sign(w_sum) # [C,C]
            elif self.co_weighted == 17:
                weight_matrix = w_neg @ w_pos.T
            else:
                raise ValueError(f"Unknown co_weighted={self.co_weighted}")

            weight_co_tensor[ID] = weight_matrix
            weighted_freq[ID] = self.mean_freq[ID]
            weighted_wt[ID] = self.mean_wt[ID]
            weighted_both[ID] = self.mean_wt_both[ID]
            weighted_any[ID] = self.mean_wt_any[ID]

        # Build binary masks
        self.common_freq_mask = (weighted_freq  >= torch.quantile(weighted_freq.view(19, -1),
                                                                  self.wt_threshold, dim=1).view(19, 1, 1)).float()
        self.common_wt_mask = (weighted_wt  >= torch.quantile(weighted_wt.view(19, -1),
                                                              self.wt_threshold, dim=1).view(19, 1, 1)).float()
        self.common_both_mask = (weighted_both >= torch.quantile(weighted_both.view(19, -1),
                                                                self.wt_threshold, dim=1).view(19, 1, 1)).float()
        self.common_any_mask = (weighted_any >= torch.quantile(weighted_any.view(19, -1),
                                                              self.wt_threshold, dim=1).view(19, 1, 1)).float()

        self.uncommon_freq_mask = (weighted_freq <= torch.quantile(weighted_freq.view(19, -1),
                                                                  self.wt_u_threshold, dim=1).view(19, 1, 1)).float()
        self.uncommon_wt_mask = (weighted_wt <= torch.quantile(weighted_wt.view(19, -1),
                                                              self.wt_u_threshold, dim=1).view(19, 1, 1)).float()
        self.uncommon_both_mask = (weighted_both <= torch.quantile(weighted_both.view(19, -1),
                                                                  self.wt_u_threshold, dim=1).view(19, 1, 1)).float()
        self.uncommon_any_mask = (weighted_any <= torch.quantile(weighted_any.view(19, -1),
                                                                self.wt_u_threshold, dim=1).view(19, 1, 1)).float()
        # Create inverses
        eps = 1e-12
        self.common_freq = self.common_freq_mask / (weighted_freq + eps) * weight_co_tensor
        self.common_wt = self.common_wt_mask / (weighted_wt + eps) * weight_co_tensor
        self.common_both = self.common_both_mask / (weighted_both + eps) * weight_co_tensor
        self.common_any = self.common_any_mask / (weighted_any + eps) * weight_co_tensor

        self.uncommon_freq = self.uncommon_freq_mask / (weighted_freq + eps) * weight_co_tensor
        self.uncommon_wt = self.uncommon_wt_mask / (weighted_wt + eps) * weight_co_tensor
        self.uncommon_both = self.uncommon_both_mask / (weighted_both + eps) * weight_co_tensor
        self.uncommon_any = self.uncommon_any_mask / (weighted_any + eps) * weight_co_tensor

        # Create LINe masks
        for ID in range(19):
            thr = torch.quantile(self.contribution_tensor[ID], self.activation_pruning)
            self.activation_mask[ID] = (self.contribution_tensor[ID] >= thr).float()
            self.weight_mask[ID] = self.weight_tensor * self.contribution_tensor[ID]
            thr_w = torch.quantile(self.weight_mask[ID], self.weight_pruning)
            self.weight_mask[ID] = (self.weight_mask[ID] >= thr_w).float()
            thr_c = torch.quantile(self.average_activations_per_class[ID], 0)                        # To include common neurons instead of combinations, can be ignored
            self.common_neurons_mask[ID] = (self.average_activations_per_class[ID] > thr_c).float()  # To include common neurons instead of combinations, can be ignored

    # --------------------------------------------------------------
    #  Visualisation
    # --------------------------------------------------------------
    def plot_maps(self, result, img_path, gt, line_map, softmax_map, logitmin_map, logitmax_map, euclidian_map, nr_act_map, coact_map1, coact_map2, coact_map3, coact_map4):
        seg_map = result[0]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        H_img, W_img = gt.shape

        dots = [self.ood_dot, self.id_dot]

        # To plot the penultimate activations. Modify the function however you want to plot it (for example uncommon to the left, or low contributing to the left)
        if self.plot_pixel_activations:
            self.plot_pixel_activation((self.ood_dot[0], self.ood_dot[1]), kind="OOD",    img_shape=(H_img, W_img))
            self.plot_pixel_activation((self.id_dot[0], self.ood_dot[1]),  kind="Normal", img_shape=(H_img, W_img))
        if self.plot_pair_ratios:
            self.plot_pair_ratio(self.ood_dot, (H_img, W_img), nbins=50)
            self.plot_pair_ratio(self.id_dot,  (H_img, W_img), nbins=50)


        # Show all heatmaps
        print("nr activations:")
        self.plotter(img, seg_map, nr_act_map, gt, dots)
        print("line:")
        self.plotter(img, seg_map, line_map, gt, dots)
        print("softmax:")
        self.plotter(img, seg_map, softmax_map, gt, dots)
        print("logitmin:")
        self.plotter(img, seg_map, logitmin_map, gt, dots)
        print("logitmax:")
        self.plotter(img, seg_map, logitmax_map, gt, dots)
        print("euclidian:")
        self.plotter(img, seg_map, euclidian_map, gt, dots)
        print("coact_bin:")
        self.plotter(img, seg_map, coact_map1, gt, dots)
        print("coact_wt:")
        self.plotter(img, seg_map, coact_map2, gt, dots)
        print("coact_both:")
        self.plotter(img, seg_map, coact_map3, gt, dots)
        print("coact_any:")
        self.plotter(img, seg_map, coact_map4, gt, dots)

    def plotter(self,
                            img: np.ndarray,
                            seg_map: np.ndarray,
                            ood_map: np.ndarray,
                            gt: np.ndarray,
                            dots: List[Tuple[int,int,str]] = None):
        """
        Creates a 2×2 figure:
          [ (1,1) RGB + seg overlay       | (1,2) raw heatmap             ]
          [ (2,1) masked heatmap (ignore)$ | (2,2) ground‐truth (ignore)$  ]

        Arguments:
            img     : H×W×3 RGB image (uint8)
            seg_map : H×W segmentation map in [0..18] that you overlay with alpha
            ood_map : H×W float array (raw OOD‐scores)
            gt      : H×W integer ground truth with {0,1,…,18, 255=ignore}
            dots    : optional list of (y, x, color) to scatter
        """

        ignore_label = 255

        # Precompute vmin/vmax for the raw heatmap
        vmin, vmax = float(ood_map.min()), float(ood_map.max())

        # Build the masked‐OOD map (ignore pixels → masked → black)
        masked_ood = np.ma.masked_where(gt == ignore_label, ood_map)
        cmap_masked = plt.cm.jet.copy()
        cmap_masked.set_bad(color="black")

        gt_only = gt.copy()

        # Mask wherever gt == 255:
        gt_masked = np.ma.masked_where(gt_only == ignore_label, gt_only)

        cmap_gt = ListedColormap(["darkgreen", "crimson"])  # 0→green, 1→red
        cmap_gt.set_bad(color="black")                       # 255→black

        plt.figure(figsize=(16, 12))

        # ---------------------------------------
        #  Panel (1,1): RGB + segmentation overlay
        # ---------------------------------------
        plt.subplot(2, 2, 1)
        plt.imshow(img)
        plt.imshow(seg_map, cmap="jet", alpha=0.4)
        if dots is not None:
            for (y, x, c) in dots:
                plt.scatter(x, y, s=10, c=c, marker="o",
                            edgecolors="white", linewidths=1, zorder=5)
        plt.axis("off")
        plt.title("RGB + Segmentation Overlay")

        # -------------------------------
        #  Panel (1,2): RAW OOD heatmap
        # -------------------------------
        plt.subplot(2, 2, 2)
        im_raw = plt.imshow(ood_map, cmap="jet", vmin=vmin, vmax=vmax)
        if dots is not None:
            for (y, x, c) in dots:
                plt.scatter(x, y, s=10, c=c, marker="o",
                            edgecolors="white", linewidths=1, zorder=5)
        plt.colorbar(im_raw, fraction=0.046, pad=0.04, format="%.2f")
        plt.axis("off")
        plt.title("Raw OOD Heatmap")

        # -----------------------------------------------
        #  Panel (2,1): OOD heatmap with ignore as black
        # -----------------------------------------------
        plt.subplot(2, 2, 3)
        im_masked = plt.imshow(masked_ood, cmap=cmap_masked, vmin=vmin, vmax=vmax)
        if dots is not None:
            for (y, x, c) in dots:
                plt.scatter(x, y, s=20, c=c, marker="o",
                            edgecolors="white", linewidths=1.5, zorder=5)
        plt.colorbar(im_masked, fraction=0.046, pad=0.04, format="%.2f")
        plt.axis("off")
        plt.title("OOD Heatmap (ignore=black)")

        # ------------------------------------
        #  Panel (2,2): Ground Truth only
        # ------------------------------------
        plt.subplot(2, 2, 4)
        im_gt = plt.imshow(gt_masked, cmap=cmap_gt, vmin=0, vmax=1)
        cbar2 = plt.colorbar(im_gt, ticks=[0, 1], fraction=0.046, pad=0.04)
        cbar2.set_label("GT label (0=ID, 1=OOD)")
        plt.axis("off")
        plt.title("Ground Truth (ignore=black)")

        plt.tight_layout()
        plt.show()

    def plot_pixel_activation(self, pixel: Tuple[int,int], kind: str,
                              img_shape: Tuple[int,int]):
        y, x = pixel; H_img, W_img = img_shape
        _, _, Hf, Wf = self.input.shape
        yf, xf = int(y*Hf/H_img), int(x*Wf/W_img)
        act = self.input[0, :, yf, xf].cpu().numpy()
        weights = (self.model.decode_head.conv_seg.weight
                   .data.squeeze(-1).squeeze(-1).cpu().numpy())
        pred_id = int(self.model_IDs_output[0, yf, xf])
        contrib = self.average_activations_per_class[pred_id].cpu().numpy()
        order   = np.argsort(contrib)
        plt.figure(figsize=(8, 4))
        plt.plot(act[order]*weights[pred_id][order], marker='o', ms=2)
        plt.title(f'{kind} pixel ({y},{x}) pred={pred_id}')
        plt.xlabel('channel rank'); plt.ylabel('activation'); plt.tight_layout(); plt.show()

    def plot_pair_ratio(self,
                        pixel: Tuple[int, int],
                        img_shape: Tuple[int, int],
                        cls:   int = None,
                        nbins: int = None):
        y_img, x_img       = pixel[:2]
        H_img, W_img       = img_shape
        _, _, Hf, Wf       = self.feature_map.shape
        yf = int(y_img * Hf / H_img)
        xf = int(x_img * Wf / W_img)

        # 2) extract feature-vector at that location
        v = self.feature_map[0, :, yf, xf].detach().cpu().numpy()   # [512]
        if cls is None:
            cls = int(self.model_IDs_output[0, yf, xf])

        # 3) raw ratio for every (i,j) pair
        raw     = np.outer(v, v).ravel()                            # [512²]
        mean_ij = self.mean_wt_any[cls].cpu().numpy().ravel()       # [512²]
        ratio   = raw / (mean_ij + 1e-9)

        # 4) order by rarity
        rarity = self.uncommon_any[cls].cpu().numpy().ravel()
        order  = np.argsort(rarity)[::-1]
        ratio_sorted = ratio[order]

        # 5) raw vs. binned plot
        if nbins is None or nbins <= 1:
            x = np.arange(ratio_sorted.size)
            y = ratio_sorted
            xlabel = "Channel-pair rank (rarest → most common)  [raw 262 144 points]"
        else:
            groups = np.array_split(ratio_sorted, nbins)
            y = np.array([g.mean() for g in groups])

            # map each bin back onto the original rank axis
            edges = np.linspace(0, ratio_sorted.size, nbins + 1)
            x     = 0.5 * (edges[1:] + edges[:-1])      # centre rank of each bin

            xlabel = "Channel-pair rank (rarest → most common)"

        # 6) draw
        plt.figure(figsize=(12, 4))
        plt.plot(x, y, lw=1)
        plt.axhline(1.0, color="k", ls="--", lw=1)
        plt.xlabel(xlabel)
        plt.ylabel(r"$(v_i v_j)\,/\,E[a_i a_j]$")
        plt.title(f"Pixel ({y_img},{x_img}), class {cls}")
        plt.tight_layout()
        plt.show()

    # --------------------------------------------------------------
    #  Metric computation
    # --------------------------------------------------------------
    def calculate_metrics(
            self,
            gt_ids: np.ndarray,
            scores: np.ndarray,
            device: str = "cuda",
            batch_size: int = 100_000,
            plot_hist: bool = False,              # ← NEW SWITCH
            nbins: int = 100,                     # ← bins for the histogram
            title: str = "OOD score distribution"
    ):
        """
        Calculates AUROC, FPR@95 % TPR and AP with TorchMetrics
        *and* (optionally) plots a histogram of the score distribution, as well
        as an average precision curve.
        """

        # ────────────────────────────────────────────────────────────────
        # 1. metrics
        g_all = torch.from_numpy(gt_ids).to(device)
        s_all = torch.from_numpy(scores).float().to(device)

        s_all = s_all - s_all.min()
        s_all = s_all / s_all.max()

        auroc = BinaryAUROC().to(device)
        ap    = BinaryAveragePrecision().to(device)
        roc   = BinaryROC().to(device)

        for i in range(0, g_all.numel(), batch_size):
            auroc.update(s_all[i:i+batch_size], g_all[i:i+batch_size].int())
            ap.update(   s_all[i:i+batch_size], g_all[i:i+batch_size].int())
            roc.update(  s_all[i:i+batch_size], g_all[i:i+batch_size].int())

        auc  = auroc.compute().item()
        ap_v = ap.compute().item()
        fpr, tpr, _ = roc.compute()
        fpr95 = fpr[tpr >= 0.95][0].item()

        print(f"AUROC={auc:.4f}  FPR95={fpr95:.4f}  AP={ap_v:.4f}")

        # ────────────────────────────────────────────────────────────────
        # histogram
        if plot_hist:
            id_scores  = scores[gt_ids == 0]
            ood_scores = scores[gt_ids == 1]

            plt.figure(figsize=(6, 4))
            plt.hist(id_scores,  bins=nbins, alpha=0.6, density=True,
                    label=f'ID  (n={len(id_scores):,})')
            plt.hist(ood_scores, bins=nbins, alpha=0.6, density=True,
                    label=f'OOD (n={len(ood_scores):,})')
            plt.xlabel("OOD score");  plt.ylabel("Density")
            plt.title(title)
            plt.legend()
            plt.tight_layout()
            plt.show()

        plot_pr = False
        if plot_pr:

            # gt_ids is a NumPy array of 0 (ID) / 1 (OOD), scores is the OOD‐score
            precision, recall, _ = precision_recall_curve(gt_ids, scores)
            ap_scalar = average_precision_score(gt_ids, scores)

            plt.figure(figsize=(6, 4))
            plt.plot(
                recall,
                precision,
                lw=1.5,
                label=f"PR curve (AP={100*ap_scalar:.2f}%)"
            )
            plt.xlabel("Recall")
            plt.ylabel("Precision")
            plt.title("Precision–Recall Curve")
            plt.legend(loc="lower left")
            plt.tight_layout()
            plt.show()

        return auc, fpr95, ap_v

    def _calculate_coactivation_score(
        self,
        labels: np.ndarray,
        common_inverse: torch.Tensor,
        uncommon_inverse: torch.Tensor,
        chunk_size: int = 4096,
        abs_logits: bool = False
    ):
        """
        Returns an OOD-heat-map where each class’s co-activation ratio
        is weighted by either its softmax probability
        or absolute logit, then summed over classes.
        """
        if self.feature_map is None:
            raise RuntimeError("feature_map not cached by hook!")

        # dimensions & device
        _, _, Hf, Wf = self.feature_map.shape
        H_img, W_img = labels.shape
        N            = Hf * Wf
        dev          = self.device
        eps          = 1e-9

        C = self.feature_map.size(1)           # e.g. 512
        K = self.model_output.size(1)          # number of classes, e.g. 19

        # Flatten feature map → [N,C], logits → [N,K], all on GPU
        feat = (
            self.feature_map
                .squeeze(0)                    # [C,Hf,Wf]
                .permute(1,2,0)                # [Hf,Wf,C]
                .reshape(-1, C)                # [N,C]
                .to(dev, non_blocking=True)
        )
        if self.make_feat_binary == True:
            feat = (feat > 0).float()

        logits_flat = (
            self.model_output
                .squeeze(0)                    # [K,Hf,Wf]
                .permute(1,2,0)                # [Hf,Wf,K]
                .reshape(-1, K)                # [N,K]
                .to(dev, non_blocking=True)
        )

        # Choose weights: softmax probs or abs(logit)
        if self.weight_mode == 'softmax':
            w_all = torch.softmax(logits_flat / self.temperature_co, dim=1)

        elif self.weight_mode == 'logits':
            # raw logits
            w_all = logits_flat

        elif self.weight_mode == 'pred':
            # Only predicted class
            pred_ids = torch.argmax(logits_flat, dim=1)                # [N]
            w_all = torch.zeros_like(logits_flat)
            w_all.scatter_(1, pred_ids.unsqueeze(1), 1.0)              # one-hot per row
        else:
            raise ValueError(f"unknown weight_mode={self.weight_mode}")

        # 3) Prepare binary masks & normalisers on GPU
        if self.inverse_convert_to_ones == True:
            M_c = (common_inverse > 0).to(dev).float()       # [K,C,C]
            M_u = (uncommon_inverse > 0).to(dev).float()     # [K,C,C]
        else:
            M_c = common_inverse.to(dev).float()       # [K,C,C]
            M_u = uncommon_inverse.to(dev).float()     # [K,C,C]
        # n_c = M_c.view(K, -1).sum(1).clamp_min(eps)   # [K] # Class normalisation that is WRONG, but can actually improve performance(cheating)
        # n_u = M_u.view(K, -1).sum(1).clamp_min(eps)   # [K] # Class normalisation that is WRONG, but can actually improve performance(cheating)

        # flatten masks for GEMM
        M_c2 = M_c.view(K*C, C)  # [K*C, C]
        M_u2 = M_u.view(K*C, C)

        ood_out = torch.empty(N, device=dev)

        # Chunked GPU loop
        for s in range(0, N, chunk_size):
            e = min(N, s + chunk_size)
            v = feat[s:e]                           # [B, C]
            B = v.size(0)

            # project v onto each class-inverse-matrix
            pc = (v @ M_c2.T).view(B, K, C)         # [B, K, C]
            pu = (v @ M_u2.T).view(B, K, C)         # [B, K, C]

            # co-activation sums
            s_c = (v.unsqueeze(1) * pc).sum(dim=2)# / n_c   # [B, K]
            s_u = (v.unsqueeze(1) * pu).sum(dim=2)# / n_u   # [B, K]

            if self.c_u_ratio == True:
                co_ratio = s_u/s_c # [B, K]
            else:
                co_ratio = s_u     # [B, K]

            # weight, sum over classes → [B], clamp
            score = (co_ratio * w_all[s:e]).sum(dim=1)                       # [B]
            if self.clips != (0, 0):
                score = torch.clamp(score, min=self.clips[0], max=self.clips[1]) # [B]
            ood_out[s:e] = score

        ood_map_gpu = ood_out.view(1, 1, Hf, Wf)          # [1,1,Hf,Wf]

        if self.max_pool != 0:
            ood_map_gpu = F.max_pool2d(ood_map_gpu, kernel_size=self.max_pool, stride=1, padding=self.max_pool//2)
        if self.co_blur_ksize != 0:
            k = self.co_blur_ksize
            σ = float(self.co_sigma)
            if σ <= 0:
                σ = 1

            coords = torch.arange(k, device=dev) - (k // 2)
            g1d = torch.exp(-coords.float().pow(2) / (2 * σ * σ))
            g1d /= g1d.sum()
            kernel2d = (g1d[:, None] @ g1d[None, :]).view(1, 1, k, k).to(dev)
            ood_map_gpu = F.conv2d(ood_map_gpu, kernel2d, padding=k // 2)

        ood_map = ood_map_gpu.squeeze().cpu().numpy()    # [Hf,Wf]
        # np.log(ood_map, out=ood_map) #if they have to big differences

        # Uncomment if you want to do divide by another map and do additional blurring after
        # ood_map = -ood_map/self.use_elsewhere_map/self.use_elsewhere_map/self.use_elsewhere_map
        # do numpy gaussian blur here

        ood_map = cv2.resize(
            ood_map,
            (W_img, H_img),                             # width, height
            interpolation=cv2.INTER_NEAREST
        )
        return ood_map


Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118


In [11]:
# notebook_or_script.py  ──────────────────────────────────────────────
import os
from typing import List, Tuple

import torch
from mmseg.apis import init_segmentor          # ← plain MMSeg import
#from calibrator import Calibrator             # ← the file you showed

# --------------------------------------------------------------------
# 1.  Collect training image / GT paths  (unchanged helper)
# --------------------------------------------------------------------
def cityscapes_paths(img_root: str, gt_root: str) -> Tuple[List[str], List[str]]:
    images, gts = [], []
    for city in sorted(os.listdir(img_root)):
        d_img, d_gt = os.path.join(img_root, city), os.path.join(gt_root, city)
        if not os.path.isdir(d_img):
            continue
        for fn in sorted(os.listdir(d_img)):
            if not fn.endswith('_leftImg8bit.png'):
                continue
            images.append(os.path.join(d_img, fn))
            label_fn = fn.replace('_leftImg8bit.png', '_gtFine_labelIds.png')
            gts.append(os.path.join(d_gt, label_fn))
    return images, gts


img_root = '/content/drive/MyDrive/leftImg8bit_trainvaltest/leftImg8bit/train'
gt_root  = '/content/drive/MyDrive/gtFine_trainvaltest/gtFine/train'
images, gts = cityscapes_paths(img_root, gt_root)
print(f'Found {len(images)} images')

# --------------------------------------------------------------------
# 2.  Bring up the MMSeg model  (one line)
# --------------------------------------------------------------------
CFG  = 'configs/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes.py'
CKPT = ('https://download.openmmlab.com/mmsegmentation/v0.5/'
        'deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/'
        'deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth')

DEV = 'cuda:0' if torch.cuda.is_available() else 'cpu'
mmseg_model = init_segmentor(CFG, CKPT, device=DEV)   # <-- raw MMSeg model

# --------------------------------------------------------------------
# 3.  Calibrate (or reload previous statistics)
# --------------------------------------------------------------------
SAVE = '/content/drive/MyDrive/calibrator_state_ground_truth.pt'

if os.path.exists(SAVE):
    # one-liner restore: builds a *fresh* model internally
    calib = Calibrator.load(SAVE, mmseg_cfg=CFG, mmseg_ckpt=CKPT, device=DEV)
else:
    calib = Calibrator(mmseg_model)    # hand the model in
    calib.run(images, gts)             # long pass over the dataset
    calib.save(SAVE)


Found 2975 images


/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from http path: https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth
load checkpoint from http path: https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth
[Calibrator] loaded ← /content/drive/MyDrive/calibrator_state_ground_truth.pt


In [ ]:
# ╔════════════════════════════════════════╗
# ║  Grid-search for LINe + Co-activation  ║
# ╚════════════════════════════════════════╝

import itertools
import gc

start, end = 0, 100     # Max 100
matched_images_short      = matched_images[start:end]
matched_annotations_short = matched_annotations[start:end]
images_to_be_plotted      = []  # show figures only for the first image if 0, can use 3:7 to plot image 4-8 for example. Empty for no figures

# Hyper-parameter lists (edit to sweep more values)
activation_clippings            = [100]
activation_prunings             = [0]
weight_prunings                 = [0]
temperatures_line               = [1]

inverse_convert_to_ones         = [False, True]
binary_or_not                   = [False, True]
wt_thresholds                   = [0, 0]
wt_u_thresholds                 = [1, 0.1]
temperatures_co                 = [0.3, 1]

baseline_and_line_blur_ksizes   = [(3, 3)]
baseline_and_line_sigmas        = [0]

co_blur_sizes                   = [23]
co_sigmas                       = [5]
max_pools                       = [11]

clips                           = [(0, 1000000), (250000, 10000000), (0, 0)]
co_weighteds                    = [0]
weight_modes                    = ['softmax']
use_c_u_ratios                  = [False, True]

plot_pixel_acts   = True
plot_pair_ratios  = True
id_dot, ood_dot   = (850, 1900, 'lime'), (485, 1360, 'red') # Place these manually by choosing coordinates on a ood object and an id object.

state = calib.state_dict()


grid = {
    "temp_co":                temperatures_co,
    "activation_clipping":    activation_clippings,
    "activation_pruning":     activation_prunings,
    "weight_pruning":         weight_prunings,
    "wt_pair":                list(zip(wt_thresholds, wt_u_thresholds)),
    "co_blur_pair":           list(zip(co_blur_sizes, co_sigmas)),
    "clip_val":               clips,
    "temp_line":              temperatures_line,
    "co_weighted":            co_weighteds,
    "inv2ones":               inverse_convert_to_ones,
    "bin_or_not":             binary_or_not,
    "baseline_pair":          list(zip(baseline_and_line_blur_ksizes, baseline_and_line_sigmas)),
    "max_pool":               max_pools,
    "weight_mode":            weight_modes,
    "c_u_ratio":              use_c_u_ratios,
}

param_names, param_values = zip(*grid.items())

# 4. Tracking best metrics
best_ap    =  -1
best_fpr   =   1
best_auc   =  -1
best_tag_ap  = best_tag_fpr = best_tag_auc = None

# 5. Iterate over the Cartesian product of all hyper-params
for combo in itertools.product(*param_values):
    P = dict(zip(param_names, combo))

    # unpack zipped pairs
    wt_thr,  wt_thr_u      = P["wt_pair"]
    co_blur_ksize, co_sigma = P["co_blur_pair"]
    baseline_blur, baseline_sigma = P["baseline_pair"]

    # create & configure detector
    det = AnomalyDetector(matched_images_short,
                          matched_annotations_short,
                          model,
                          state)
    det.plot_pixel_activations = plot_pixel_acts
    det.plot_pair_ratios       = plot_pair_ratios
    det.images_to_be_plotted   = images_to_be_plotted
    det.id_dot, det.ood_dot    = id_dot, ood_dot

    # set thresholds & params
    det.activation_clipping     = P["activation_clipping"]
    det.activation_pruning      = P["activation_pruning"]
    det.weight_pruning          = P["weight_pruning"]
    det.temperature_co          = P["temp_co"]
    det.temperature_line        = P["temp_line"]
    det.inverse_convert_to_ones = P["inv2ones"]
    det.make_feat_binary        = P["bin_or_not"]
    det.wt_threshold            = wt_thr
    det.wt_u_threshold          = wt_thr_u
    det.co_blur_ksize           = co_blur_ksize
    det.co_sigma                = co_sigma
    det.blur_ksize              = baseline_blur
    det.sigma                   = baseline_sigma
    det.max_pool                = P["max_pool"]
    det.clips                   = P["clip_val"]
    det.co_weighted             = P["co_weighted"]
    det.weight_mode             = P["weight_mode"]
    det.c_u_ratio               = P["c_u_ratio"]

    # run inference
    det.ood_inference()
    IDs, ood_scores = det.get_ood_score_lists()

    # evaluate selected methods
    methods = {
        "ood_scores_coact_bin":       "Co-act count",
        "ood_scores_coact_wt":        "Co-act magnitude",
        "ood_scores_coact_any":       "Co-act any",
        "ood_scores_nr_activation":   "#activations",
    }

    # build a reusable run tag
    base_tag = " | ".join(f"{k}={P[k]}" for k in param_names)

    scores_line = ood_scores["ood_scores_line"]
    for method, title in methods.items():
        scores = -ood_scores[method] / scores_line / scores_line / scores_line
        scores = (scores - scores.min()) / (scores.ptp() + 1e-9)

        run_tag = f"method | ={method}{base_tag}"
        print(run_tag)

        auc, fpr, ap = det.calculate_metrics(IDs, scores,
                                             plot_hist=False,
                                             title=title)

        if ap  > best_ap:  best_ap,  best_tag_ap  = ap,  run_tag
        if fpr < best_fpr: best_fpr, best_tag_fpr = fpr, run_tag
        if auc  > best_auc: best_auc, best_tag_auc = auc, run_tag

    # cleanup GPU memory
    del det
    gc.collect()
    torch.cuda.empty_cache()

# 6. Print summary of best combinations
print("\n══════ BEST RESULTS ══════")
print(f"• Best AP    = {best_ap:.4f}  → {best_tag_ap}")
print(f"• Best FPR95 = {best_fpr:.4f}  → {best_tag_fpr}")
print(f"• Best AUROC = {best_auc:.4f}  → {best_tag_auc}")


method | =ood_scores_coact_bintemp_co=0.3 | activation_clipping=100 | activation_pruning=0 | weight_pruning=0 | wt_pair=(0, 1) | co_blur_pair=(23, 5) | clip_val=(0, 1000000) | temp_line=1 | co_weighted=0 | inv2ones=False | bin_or_not=False | baseline_pair=((3, 3), 0) | max_pool=11 | weight_mode=softmax | c_u_ratio=False
AUROC=0.9501  FPR95=0.3238  AP=0.2127
method | =ood_scores_coact_wttemp_co=0.3 | activation_clipping=100 | activation_pruning=0 | weight_pruning=0 | wt_pair=(0, 1) | co_blur_pair=(23, 5) | clip_val=(0, 1000000) | temp_line=1 | co_weighted=0 | inv2ones=False | bin_or_not=False | baseline_pair=((3, 3), 0) | max_pool=11 | weight_mode=softmax | c_u_ratio=False
AUROC=0.9575  FPR95=0.1893  AP=0.2036
method | =ood_scores_coact_anytemp_co=0.3 | activation_clipping=100 | activation_pruning=0 | weight_pruning=0 | wt_pair=(0, 1) | co_blur_pair=(23, 5) | clip_val=(0, 1000000) | temp_line=1 | co_weighted=0 | inv2ones=False | bin_or_not=False | baseline_pair=((3, 3), 0) | max_pool=11

In [15]:
# Set true and run if you need to free memory

delete = True
if delete:
    del det
    import gc
    gc.collect()
    import torch
    torch.cuda.empty_cache()